# Python Numeric Data Analysis - Pandas

* For the times when you have Structured data, CSV, spreadsheets, R dataframes, SQL tables.

* Like numpy tables but more sophisticated labelling of rows and columns.  Good at dealing with missing and messy data. Heterogeneous data types. Time series data.

* Clean up and explore data, prepare it for analysis.

* Analyse or pass on to other systems (Scikit-learn, tensorflow, etc)

In [ ]:
# The usual suspects
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 8]

# And pandas
import pandas as pd


## Understanding the Pandas classes

### Series

Like a numpy array but with index labels:

In [ ]:
s = pd.Series([1.1, 2.2, 3.3, 4.4])
s

In [ ]:
s[1:3]

In [ ]:
s = pd.Series([1.1, 2.2, 3.3, 4.4], index=["alice", "bob", "charles", "diana"])
s

In [ ]:
s['charles']

In [ ]:
s['alice':'charles':2]

In [ ]:
s.index

In [ ]:
s.values

We can construct a Series from, e.g., a Python dict:

In [ ]:
populations = pd.Series({
    "London":    8173941,
    "Birmingham":1085810,
    "Glasgow":    590507,
    "Liverpool":  552267, 
    "Bristol":    535907
})
populations

And we can use it a lot like a Python dict...

In [ ]:
for k in populations.keys():
    print(k)

In [ ]:
"Bristol" in populations

but with ordering, and the power of numpy arrays:

In [ ]:
populations / 1_000_000

In [ ]:
(populations / 1_000_000).std()

In [ ]:
populations.idxmax()

In [ ]:
populations.sort_index()

In [ ]:
populations.sort_values()

In [ ]:
populations[populations > 1_000_000]

c.f.
`SELECT index, value FROM populations WHERE value > 1000000;`

Here's a different dataset: the percentage of men in the population:

In [ ]:
male_percent = pd.Series({
    "London": 49.12,
    "Birmingham": 49.42,
    "Leeds": 49.43,
    "Glasgow": 47.73,
    "Bristol": 49.59,
})
male_percent

Percentage of women:

In [ ]:
100 - male_percent

In [ ]:
female_pops = populations * (100 - male_percent) / 100.0
female_pops

Note that a few clever things happened there!

In [ ]:
female_pops[ female_pops.notnull() ].astype(int)

If we had reason to believe there were *no* women in the cities for which we had incomplete data:

In [ ]:
female_pops.fillna(0).astype(int)

All pretty clever, but that's just the **Series**.  There's a more powerful class...

---

### DataFrames

![Dataframe](dataframe.png)

## A quick look at a DataFrame

In [ ]:
df = pd.read_excel("landmarks.xls", sheet_name="landmarks")
df

In [ ]:
df.head(7)

In [ ]:
df.columns

In [ ]:
df.index

### Columns come first

It's *important* to appreciate that the first axis in a dataframe selects *columns*, not *rows*.

In [ ]:
df['confidence']

In [ ]:
type(df['confidence'])

In [ ]:
df['confidence'].values.mean()

In [ ]:
df['confidence'].mean()

In [ ]:
np.mean(df['confidence'])

In [ ]:
df['confidence'][23:26]

**Important:** Note that this output is still a Series, so you need to respect its indexing:

In [ ]:
df['confidence'][23:26][23]

If the column name is suitable, you can refer to it as an attribute:

In [ ]:
df.confidence.max()

In [ ]:
df[ ['frame', 'timestamp', 'confidence'] ].head()

In [ ]:
df.describe()

We can extract sections of the dataframe using slices:

In [ ]:
df[400:404]

Note that _the result is also a DataFrame_.

We can also use boolean indexes in Pandas structures:

In [ ]:
df.confidence > 0.96

In [ ]:
df[df.confidence > 0.96]

---

## A potential source of confusion

So the dataframe is indexed first by the columns:

    df['confidence']
    
If you treat it like a Python dict, the keys are the column names:

In [ ]:
for i in df.keys():
    print(i)

Simple indexing into a dataframe will give you a column.

But slices, boolean indexes etc, as we've seen above, give you a dataframe.  

It's not unusual for different types of indexes to give you different things -- slicing a list in Python gives you a list, simple indexing gives you an entry from the list.

If you're used to Numpy arrays or Pandas Series, note that you can't normally do:

    df[0]
    
with a Dataframe... unless there happens to be a *column* named '0'.

This will never give you a row.

If you do want to get a *row* by number, there's an attribute for that:

In [ ]:
df.iloc[2]  # returns a Series for the row

In [ ]:
df.iloc[2]['timestamp']

**Question:** What will this do?

In [ ]:
for p in df[:5]:
    print(p)

OK, are we happy?

---

# Some sample data - the Lab Weather Station

![station](https://www.cl.cam.ac.uk/research/dtg/weather/images/wgb-view.jpg)

The lab has a weather station at https://www.cl.cam.ac.uk/research/dtg/weather/.

![dials](https://www.cl.cam.ac.uk/research/dtg/weather/images/current-dials.png?)

The data is collated into various downloadable files:

In [ ]:
df = pd.read_csv('https://www.cl.cam.ac.uk/research/dtg/weather/weather-raw.csv', header=None)
df

Note that if the columns don't have names as headers, they'll be given numbers:

In [ ]:
df[0]

Let's give the columns names, and parse the timestamps:

In [ ]:
df = pd.read_csv(
    'https://www.cl.cam.ac.uk/research/dtg/weather/weather-raw.csv', 
    names=[
        'timestamp', 'temp_dc','humidity', 'dewpoint_dc', 'pressure_mbar', 
        'mean_wind_speed_dk', 'ave_wind_bearing', 'sunshine_ch', 'rainfall_um', 'max_wind_dk'
    ],
    parse_dates=['timestamp']
)

df.head(10)

`read_csv()` is a very powerful function.

In [ ]:
pd.read_csv?

See also `pd.read_json()`, `pd.read_xml()`, `pd.read_hdf5()` etc, and the related `to_` output functions.

In [ ]:
basics = df[['timestamp', 'temp_dc', 'rainfall_um']]
print(
    basics[0:10].to_csv(sep='|')
)

In [ ]:
df.dtypes

In [ ]:
df.timestamp.head()

DataFrames have a `plot()` method:

In [ ]:
df.temp_dc[:10000].plot();
# Or you can do plt.plot(df.temp_dc[:10000])

You can easily create new columns:

In [ ]:
df['temp']            = df['temp_dc'] / 10.0
df['dewpoint']        = df['dewpoint_dc'] / 10.0
df['mean_wind_kts']   = df['mean_wind_speed_dk'] / 10.0
df['max_wind_kts']    = df['max_wind_dk'] / 10.0
df['sunshine_hours']  = df['sunshine_ch'] / 100.0
df['rainfall_mm']     = df['rainfall_um'] / 1000.0
df.tail()

For us, the time is more useful as an index:

In [ ]:
df.set_index('timestamp', inplace=True)
df

In [ ]:
df.index

Note 'inplace'. Some methods have names like 'set_index' but don't change the original by default.

In [ ]:
df['humidity']['2018-11-08 21:30:00']

In [ ]:
df['humidity'][datetime(year=2018, month=11, day=8, hour=21, minute=30, second=0)]

We can also request the row, and then ask for its humidity component.  If we aren't asking for it by its integer position, we use `loc()` instead of `iloc()`.

In [ ]:
df.loc['2018-11-08 21:30:00']['humidity']

Let's plot the values in a column.

In [ ]:
df.temp.plot();

Looks as if there are some dodgy readings there.  Let's zoom in:

In [ ]:
df['2015-09-01':'2016-01-31']["temp"].plot();

In [ ]:
dodgy_rows =  df['2015-10-22':'2016-01-10'].index

df = df.drop(dodgy_rows)
df["temp"].plot();

In [ ]:
df[["temp", "rainfall_mm"]].plot();

In [ ]:
print(df.temp.max(), df.temp.idxmax())
print(df.temp.min(), df.temp.idxmin())

In [ ]:
print(df['temp'].quantile(0.99))
print(df['temp'].quantile(0.01))

In [ ]:
df.temp.idxmin() - df.temp.idxmax()

In [ ]:
df.info()

In [ ]:
df.plot(kind='scatter', x='mean_wind_kts', y='temp');

We can do some other interesting things with a time-based index:

In [ ]:
df['temp'][df.index.hour == 0][5000:7000].plot()
df['temp'][df.index.hour == 14][5000:7000].plot();

# GroupBy

In [ ]:
df.index.month

In [ ]:
monthgrouper = df.groupby(df.index.month)
monthgrouper

Not very helpful - what's happening behind the scenes?

In [ ]:
monthgrouper.groups

In [ ]:
july_frame = df.loc[monthgrouper.groups[7]]
july_frame.mean()

Maybe plot the first 10000...

In [ ]:
for month, groupframe in monthgrouper:
    groupframe['temp'][:10000].plot(style='.', label=month)
plt.legend();

You can call some methods directly on the groupby object:

In [ ]:
monthgrouper.mean()

In [ ]:
for month, groupframe in monthgrouper:
    print("month:",month)
    groupframe.plot(x='temp', y='mean_wind_kts', kind='scatter', xlim=[-10,35], ylim=[0, 50])
    plt.show()

In [ ]:
hourgrouper = df.groupby(df.index.hour)
rains = { hour: frame["rainfall_mm"].mean() * 2 for hour, frame in hourgrouper }
s = pd.Series(rains)
s.plot(title="Cambridge rainfall throughout the day");

So Pandas is good for:

* reading data
* cleaning data
* merging data
* exploring data
* understanding data

and especially valuable as a first stage before using data for anything else.

---


# That's all for now!

These notebooks are available at [http://github.com/quentinsf/labnotebook](http://github.com/quentinsf/labnotebook).

I suggest you Google for '[A gallery of interesting Jupyter Notebooks](https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks)'